In [ ]:
import math
import unittest
import numpy as np


def translate(new_center):
    return np.array([
            [1, 0, -new_center[0]],
            [0, 1, -new_center[1]],
            [0, 0, 1]])


def rotate(alpha):
    return np.array([
            [ np.cos(alpha), np.sin(alpha), 0],
            [-np.sin(alpha), np.cos(alpha), 0],
            [ 0,             0,             1]])


class CoordinateSystem:
    def __init__(self, center, alpha):
        self.center = center
        self.alpha = alpha
        self.transform = rotate(alpha) @ translate(center)
    
    def global_to_local(self, pt):
        pt_g = np.array([pt[0], pt[1], 1])
        pt_l = self.transform @ pt_g
        return tuple(pt_l[:-1])


class CoordinateSystemTest(unittest.TestCase):
    
    def test_translation_only(self):
        local_system = CoordinateSystem((1, 1), 0)
        global_point = (3, 2)
        local_point = local_system.global_to_local(global_point)
        self.assertEqual((2, 1), local_point)
        
    def test_rotation_only(self):
        local_system = CoordinateSystem((0, 0), math.pi / 4)
        global_point = (1, 1)
        local_x, local_y = local_system.global_to_local(global_point)
        self.assertAlmostEqual(np.sqrt(2), local_x)
        self.assertAlmostEqual(0, local_y)
        
    def test_rotation_and_translation(self):
        local_system = CoordinateSystem((1, 1), math.pi / 4)
        global_point = (2, 2)
        local_x, local_y = local_system.global_to_local(global_point)
        self.assertAlmostEqual(np.sqrt(2), local_x)
        self.assertAlmostEqual(0, local_y)

In [ ]:
class BicycleModel: 
    def __init__(self, frame_length):
        self.frame_length = frame_length
    
    def predict_trajectory(self, velocity, steer_angle, dt=0.2, n_points=50):
        states = [(0.0, 0.0, 0.0)]
        for _ in range(n_points-1):
            next_state = self._next_state(states[-1], velocity, steer_angle, dt)
            states.append(next_state)
        return [(x, y) for x, y, _ in states]
    
    def _next_state(self, state, velocity, steer_angle, dt):
        x, y, psi = state
        x_next = x + velocity * np.cos(psi) * dt
        y_next = y + velocity * np.sin(psi) * dt
        psi_next = psi + velocity / self.frame_length * steer_angle * dt
        return (x_next, y_next, psi_next)
    
    
class BicycleModelTest(unittest.TestCase):
    
    def test_no_motion(self):
        model = BicycleModel(frame_length=1)
        trajectory = model.predict_trajectory(velocity=0, steer_angle=0, dt=0.1, n_points=2)
        self.assertEqual([(0, 0), (0, 0)], trajectory)
        
    def test_linear_motion(self):
        model = BicycleModel(frame_length=1)
        trajectory = model.predict_trajectory(velocity=2, steer_angle=0, dt=0.1, n_points=2)
        self.assertEqual([(0, 0), (0.2, 0)], trajectory)
        
    def test_make_cicrle(self):
        model = BicycleModel(frame_length=2)
        one_degree_steer = math.pi / 180
        trajectory = model.predict_trajectory(velocity=2, steer_angle=one_degree_steer, dt=0.1, n_points=3601)
        last_x, last_y = trajectory[-1]
        self.assertAlmostEqual(0, last_x)
        self.assertAlmostEqual(0, last_y)
        
unittest.main(argv=[''], exit=False)        

In [ ]:
import re
import json
import traceback as tb

from simple_websocket_server import WebSocketServer, WebSocket

def mph2mps(speed):
    return 1609.34 * speed / 3600

class TelemetryPackage:
    def __init__(self, data):
        self.__data = data
        
    @property
    def center(self):
        return (self.__data['x'], self.__data['y'])
    
    @property
    def heading(self):
        return self.__data['psi']
    
    @property
    def velocity(self):
        return mph2mps(self.__data['speed'])
    
    @property
    def steering_angle(self):
        return -self.__data['steering_angle']
    
    @property
    def trajectory(self):
        return [(x, y) for x, y in zip(self.__data['ptsx'], self.__data['ptsy'])]
    
class SteerPackage: 
    def __init__(self, throttle, steering_angle):
        self.throttle = throttle
        self.steering_angle = steering_angle
        self.planned_trajectory = [(0, 0)]
        self.predicted_trajectory = [(0, 0)]
        
    def to_json(self):
        next_x, next_y = self._unzip(self.planned_trajectory)
        mpc_x, mpc_y = self._unzip(self.predicted_trajectory)
        return json.dumps({
            'throttle': self.throttle, 
            'steering_angle': self.steering_angle, 
            'mpc_x': mpc_x,
            'mpc_y': mpc_y,
            'next_x': next_x, 
            'next_y': next_y
        })
    
    def _unzip(self, tuples):
        return tuple(zip(*tuples))
    

class Controller: 
    def process(self, telemetry):
        return SteerPackage(throttle=0.05, steering_angle=-0.02)
    
    
def planned_trajectory(telemetry):
    car_coord_system = CoordinateSystem(telemetry.center, telemetry.heading)
    return [car_coord_system.global_to_local(pt) for pt in telemetry.trajectory]

class SimulatorClient(WebSocket):
    incoming_pattern = re.compile('42\["telemetry",\s*(.*)\s*\]')
    manual_command = '42["manual",{}]'
    steer_template = '42["steer",%s]'
    
    def connected(self):
        self.vehicle_model = BicycleModel(frame_length=2.67)
        self.controller = Controller()
        
    def handle(self):
        try: 
            telemetry = self._receive_telemetry(self.data)
            if not telemetry: 
                self._send_manual()
            else:
                steer_command = self.controller.process(telemetry)
                self._calc_trajectories(steer_command, telemetry)
                self._send_streer(steer_command)
        except Exception as ex:
            tb.print_exc()
    
    def handle_close(self):
        pass
    
    def _calc_trajectories(self, command, telemetry):
        command.planned_trajectory = planned_trajectory(telemetry)
        command.predicted_trajectory = self.vehicle_model.predict_trajectory(
            telemetry.velocity, telemetry.steering_angle, dt=2, n_points=10)
    
    def _receive_telemetry(self, data):
        match_data = self.incoming_pattern.match(data)
        if not match_data: return None
        
        telemetry_data = json.loads(match_data.group(1))
        return TelemetryPackage(telemetry_data)
    
    def _send_manual(self):
        self.send_message(self.manual_command)
        
    def _send_streer(self, command):
        response = self.steer_template % command.to_json()
        self.send_message(response)
        

class UdacitySimulator: 
    
    def run(self):
        server = WebSocketServer('', 4567, SimulatorClient)
        try:
            server.serve_forever()
        except KeyboardInterrupt: 
            server.close()


## Simulator runner

In [ ]:
sim = UdacitySimulator()
sim.run()